# Курсовой проект по дисциплине "Численные методы"
# Тема: "Метод бисопряженных градиентов."

## Подготовила студентка группы М8О-307 Алексюнина Юлия
## Руководитель: Ревизников Д.Л.

### Постановка задачи

Реализовать решение систем линейных алгебраических уравнений с 
несимметричными разреженными матрицами большой размерности методом бисопряженных градиентов.


### Описание метода

    Создадим случайную матрицу, поставим плотность (density) = 0.6, таким образом, наша матрица будет на 40% состоять из нулей. Остальные значения будут находиться в диапазоне от 0 до 1. Можно выбрать размер матрицы от 4 до n.

    Рассмотрим СЛАУ с вещественными коэффициентами(Ax=b). В основу алгоритма ложится идея проекционного метода и использование свойства A-бисопряженности системы векторов, а именно: векторы  p – невязки бисопряжённые, если скалярные произведения (Api,pj)=0, (Api,pj)=0. Фактически, данное условие эквивалентно биортогональности относительно энергетического скалярного произведения.
    
    Общая концепция проекционных методов предписывает нам выбрать два подпространства. В нашем случае подпространства мы выберем два подпространства, задаваемые матрицей системы А, вектором невязки на нулевой итерации.

    Метод основан на построении биортогонального базиса p пространства  K_k(A,r0) K_k(A,r0) и вычислении поправки такой, что новое приближение на очередной итерации было бы ортогонально второму подпространству Крылова. Базисные вектора строятся до тех пор, пока не будет достигнут критерий остановки итерационного процесса, а каждое последующее приближение формируется, как сумма приближения на предыдущей итерации и найденной поправки. 
    
    Критерием останова итерационного процесса является достижение невязкой значения, которое меньше некоторого наперед заданного эпсилон.


### Пример теста

![Тест11](https://sun9-68.userapi.com/impf/CH6PryquaKlefjV4ZO4e50egfe9txRQ5Q6mlrw/Jq6Ae8i7uD4.jpg?size=663x484&quality=96&proxy=1&sign=a27eb9cb1438c9179bd2f88689d89c5c&type=album)
![Тест12](https://sun9-14.userapi.com/impf/QNFww4YgWiYQl6twpot8RiAoUTvpNKPcdjz3AQ/UwFWcfN8lOs.jpg?size=485x258&quality=96&proxy=1&sign=da2c866595ca418d161b57a5151c239d&type=album)

### Решение

#### Генерация матрицы. 
Импортируем необходимые библиотеки.


In [11]:
import numpy as np
from random import randint
from scipy.sparse import rand
import numpy as np
from numpy.linalg import norm
from scipy.sparse import diags, csc_matrix
from scipy.sparse.linalg import bicgstab, spilu 

Сгенерируем матрицу заданного размера с плотностью 0.6 и вектор правой части.

In [12]:
shape = int(input())
if shape < 3:
        exit()

matrix = rand(shape, shape, density=0.6, random_state=randint(112, 154))


for i in matrix.toarray().round(3):
    for j in i:
        print(f'{j} ', end=" ")
    print('\n')
print('\n')
b = np.random.randint(5, 53, shape)
for i in b:
    print(f'{i} ', end=" ")
print('\n')
print(type(matrix))

10
0.0  0.465  0.0  0.0  0.0  0.052  0.201  0.0  0.368  0.567  

0.151  0.036  0.609  0.0  0.229  0.72  0.977  0.592  0.0  0.809  

0.867  0.64  0.378  0.607  0.33  0.0  0.0  0.087  0.618  0.508  

0.846  0.0  0.736  0.0  0.163  0.972  0.0  0.0  0.0  0.087  

0.611  0.327  0.0  0.0  0.375  0.0  0.0  0.246  0.183  0.0  

0.506  0.004  0.0  0.0  0.744  0.269  0.991  0.0  0.0  0.0  

0.0  0.187  0.146  0.23  0.0  0.046  0.0  0.23  0.0  0.0  

0.0  0.648  0.923  0.89  0.0  0.159  0.457  0.0  0.0  0.657  

0.846  0.915  0.0  0.505  0.136  0.0  0.006  0.005  0.0  0.181  

0.083  0.156  0.0  0.0  0.895  0.549  0.0  0.901  0.583  0.0  



23  27  23  52  13  22  43  42  20  30  

<class 'scipy.sparse.coo.coo_matrix'>


#### Реализация алгоритма

![Алгоритм](https://sun9-31.userapi.com/impf/eB8P4H6U7fR28U9YD9XuwvE1emlksS9FQhuEfg/zalHp3WbPW0.jpg?size=370x441&quality=96&proxy=1&sign=dc07ec63bc662226176c3a6a42efafc0)

Критерий остановки итерационного процесса: норма заданной невязки меньше эпсилон.

Представим исходную матрицу в виде сжатой матрицы разреженных столбцов.

In [13]:
matrix = csc_matrix(matrix)

Создадим класс BiCGMethod, в котором реализуем несколько методов. 
    __init__ - для инициализации необходимых данных;
    solve - подготовка перед итерационным процессом и сам процесс решения;
    print_solution - печать результата и сравнение его с решением при помощи функций встроенной библиотеки Numpy

In [14]:
class BiCGMethod:
    def __init__(self, matrix, b, x0=None, eps=1e-5):
        self.matrix = matrix
        self.b = b
        self.eps = eps
        self.shape = matrix.shape[0]
        self.x0 = np.array([0] * self.shape) if x0 is None else x0
        self.k = 0
        
    def solve(self):
        r0 = self.b - self.matrix @ self.x0 # невязка
        x0 = self.x0 # начальное приближение
        r2 = r0 # выбирается вектор
        rho0 = 1
        alpha0 = 1
        omega0 = 1
        v0 = np.array([0] * self.shape)
        p0 = np.array([0] * self.shape)
        while True:
            rho = r2 @ r0
            beta = (rho * alpha0) / (rho0 * omega0)
            p = r0 + beta * (p0 - omega0 * v0)
            v = self.matrix @ p
            alpha = rho / (r2 @ v)
            s = r0 - alpha * v
            t = self.matrix @ s
            omega = (t @ s) / (t @ t)
            x = x0 + omega * s + alpha * p
            r = s - omega * t


            self.k += 1
            if norm(r) < self.eps: # норма заданной невязки
                break
            r0 = r
            rho0 = rho
            alpha0 = alpha
            omega0 = omega
            v0 = v
            p0 = p
            x0 = x
        return x
    
    def print_solution(self):
        x = self.solve()
        x2 = np.linalg.solve(self.matrix.toarray(), self.b)
        # with open(self.output, 'w') as f:
        print('My solve:\n')
        print(f'{x.round(5)}\n')
        print(f'EPS = {self.eps}\n')
        print(f'Shape = {self.shape}\n')
        print(f'Count of iterations = {self.k}\n')
        # print(f'Mean = {np.mean(x)}\n') # среднее
        print('\nNumPy solve:\n')
        print(f'{x2.round(5)}\n')
        # print(f'Mean = {np.mean(x2)}\n')


Наконец, сделаем расчет.

In [15]:
solver = BiCGMethod(matrix, b, eps=1e-5)
solver.print_solution()

My solve:

[  -1.52111  124.80777   88.01166  -34.88747 -212.07618   47.51687
  168.77658   55.19391  213.7142  -264.70185]

EPS = 1e-05

Shape = 10

Count of iterations = 12


NumPy solve:

[  -1.52111  124.80777   88.01166  -34.88747 -212.07619   47.51687
  168.77658   55.19391  213.7142  -264.70185]



### Вывод
В ходе данной лабораторной работы был изучен алгоритм бисопряженных градиентов. Этот алгоритм хорошо зарекомендовал себя для решения систем линейных алгебраический уравнений, поскольку обладает некоторыми важными свойствами: он численно устойчив; не меняется вид матрицы в процессе решения; эффективен для решения систем большой размерности с разреженной матрицей, поскольку в методе самая трудоемкая операция - умножение матрицы на вектор; применим для решения систем с несимметричными матрицами. Вычислительная мощность:  N/4